In [1]:
import pandas as pd
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import sqlalchemy
import h5py

In [2]:
db_connection_string = 'sqlite:///./Resources/product.db'
engine  = sqlalchemy.create_engine(db_connection_string)

inspector = sqlalchemy.inspect(engine)
table_names = inspector.get_table_names()
print(table_names)

['AAPL_1_Day_Candles', 'AAPL_1_Min_Candles', 'AAPL_Indicators', 'AAPL_Info', 'MSFT_1_Day_Candles', 'MSFT_1_Min_Candles', 'MSFT_Indicators', 'MSFT_Info']


In [3]:
ticker = 'MSFT'
indicators_df = pd.read_sql_table(ticker + '_Indicators', con=engine, index_col='Datetime')

---

## Prepare the data to be used on a neural network model

In [4]:
print(indicators_df.head())

                       Open    High     Low   Close  Volume  CDLDOJI  \
Datetime                                                               
2021-09-07 05:42:00  300.65  300.65  300.60  300.60    1730        0   
2021-09-07 05:43:00  300.65  300.65  300.59  300.59    3386        0   
2021-09-07 05:46:00  300.61  300.61  300.59  300.59     415        0   
2021-09-07 05:53:00  300.60  300.60  300.60  300.60     185      100   
2021-09-07 05:54:00  300.61  300.61  300.61  300.61     295      100   

                     CDLHOMINGPIGEON  CDLSPINNINGTOP  Trade Signal  \
Datetime                                                             
2021-09-07 05:42:00                0               0           0.0   
2021-09-07 05:43:00                0               0           0.0   
2021-09-07 05:46:00                0               0           0.0   
2021-09-07 05:53:00                0               0           1.0   
2021-09-07 05:54:00                0               0           1.0   

    

In [5]:
# Review the data types associated with the columns
indicators_df.dtypes

Open               float64
High               float64
Low                float64
Close              float64
Volume               int64
CDLDOJI              int64
CDLHOMINGPIGEON      int64
CDLSPINNINGTOP       int64
Trade Signal       float64
Upper Band         float64
Middle Band        float64
Lower Band         float64
EMA                float64
MACD               float64
MACD_SIGNAL        float64
MACD_HIST          float64
AD                 float64
ADOSC              float64
OBV                float64
ATR                float64
NATR               float64
TRANGE             float64
WCLPRICE           float64
HT_DCPERIOD        float64
LINEARREG          float64
VAR                float64
dtype: object

In [6]:
display(indicators_df['Trade Signal'].value_counts())

 0.0    7745
 1.0    2772
-1.0     198
Name: Trade Signal, dtype: int64

### Step 5: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “IS_SUCCESSFUL”. The remaining columns should define the features dataset. 



### Step 3: Encode the dataset’s categorical variables using `OneHotEncoder`, and then place the encoded variables into a new DataFrame.

In [7]:
# Create a list of categorical variables 
categorical_variables = ['Trade Signal']

# Display the categorical variables list
display(categorical_variables)

['Trade Signal']

In [8]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)


In [9]:
# Encode the categorcal variables using OneHotEncoder
encoded_data = enc.fit_transform(indicators_df[categorical_variables])


In [10]:
# Create a DataFrame with the encoded variables
encoded_df = pd.DataFrame(
    encoded_data,
    columns = enc.get_feature_names(categorical_variables)
)
# Review the DataFrame
display(encoded_df.head(10))

,Trade Signal_-1.0,Trade Signal_0.0,Trade Signal_1.0
0,0.0,1.0,0.0
1,0.0,1.0,0.0
2,0.0,1.0,0.0
3,0.0,0.0,1.0
4,0.0,0.0,1.0
5,0.0,0.0,1.0
6,0.0,0.0,1.0
7,0.0,0.0,1.0
8,0.0,0.0,1.0
9,0.0,0.0,1.0


In [11]:
encoded_df.rename(columns={'Trade Signal_-1.0': 'Bearish', 'Trade Signal_0.0': 'None', 'Trade Signal_1.0':'Bullsih'}, inplace=True)
encoded_df.drop(columns='None', inplace=True)
print(encoded_df.head())

   Bearish  Bullsih
0      0.0      0.0
1      0.0      0.0
2      0.0      0.0
3      0.0      1.0
4      0.0      1.0


### Step 4: Add the original DataFrame’s numerical variables to the DataFrame containing the encoded variables.

> **Note** To complete this step, you will employ the Pandas `concat()` function that was introduced earlier in this course. 

In [12]:
# Define the target set y using the IS_SUCCESSFUL column
y = encoded_df.copy()

# Display a sample of y
display(y.head())

,Bearish,Bullsih
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,1.0
4,0.0,1.0


In [13]:
# Define features set X by selecting all columns but IS_SUCCESSFUL
X = indicators_df.drop(columns=['Trade Signal'])

# Review the features DataFrame
display(X.head())

,Open,High,Low,Close,Volume,CDLDOJI,CDLHOMINGPIGEON,CDLSPINNINGTOP,Upper Band,Middle Band,...,AD,ADOSC,OBV,ATR,NATR,TRANGE,WCLPRICE,HT_DCPERIOD,LINEARREG,VAR
Datetime,,,,,,,,,,,,,,,,,,,,,
2021-09-07 05:42:00,300.65,300.65,300.60,300.60,1730,0,0,0,300.668990,300.620,...,-22837.976744,-1725.622336,-10037.0,0.170864,0.056841,0.05,300.6125,17.211765,300.714857,0.000600
2021-09-07 05:43:00,300.65,300.65,300.59,300.59,3386,0,0,0,300.670764,300.618,...,-26223.976744,-2805.550909,-13423.0,0.162945,0.054208,0.06,300.6050,18.893411,300.644857,0.000696
2021-09-07 05:46:00,300.61,300.61,300.59,300.59,415,0,0,0,300.672000,300.616,...,-26638.976744,-3124.335243,-13423.0,0.152735,0.050812,0.02,300.5950,20.539088,300.630857,0.000784
2021-09-07 05:53:00,300.60,300.60,300.60,300.60,185,100,0,0,300.650900,300.606,...,-26638.976744,-2970.716539,-13238.0,0.142539,0.047418,0.01,300.6000,22.069379,300.622571,0.000504
2021-09-07 05:54:00,300.61,300.61,300.61,300.61,295,100,0,0,300.612967,300.598,...,-26638.976744,-2637.807384,-12943.0,0.133072,0.044267,0.01,300.6100,23.446923,300.619143,0.000056


### Step 6: Split the features and target sets into training and testing datasets.


In [14]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

### Step 7: Use scikit-learn's `StandardScaler` to scale the features data.

In [15]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


---

## Compile and Evaluate a Binary Classification Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [16]:
# Define the the number of inputs (features) to the model
number_input_features = len(list(X.columns))
# Review the number of features
number_input_features


25

In [17]:
# Define the number of neurons in the output layer
number_output_neurons = 2

In [18]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 =  int(round((number_input_features + number_output_neurons)/2, 0))
# Review the number hidden nodes in the first layer
hidden_nodes_layer1


14

In [19]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 =  int(round((hidden_nodes_layer1 + number_output_neurons)/2, 0))

# Review the number hidden nodes in the second layer
hidden_nodes_layer2


8

In [20]:
# Create the Sequential model instance
nn = Sequential()


In [21]:
# Add the first hidden layer
nn.add(Dense(units=hidden_nodes_layer1, activation="relu", input_dim=number_input_features))


In [22]:
# Add the second hidden layer
nn.add(Dense(units=hidden_nodes_layer2, activation="relu"))


In [23]:
# Add the output layer to the model specifying the number of output neurons and activation function
nn.add(Dense(units=number_output_neurons, activation="sigmoid"))


In [24]:
# Display the Sequential model summary
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 14)                364       
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 120       
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 18        
Total params: 502
Trainable params: 502
Non-trainable params: 0
_________________________________________________________________


### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [25]:
# Compile the Sequential model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [26]:
# Fit the model using 50 epochs and the training data
model = nn.fit(X_train_scaled, y_train, epochs=50, verbose=0)


### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [27]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=True)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

84/84 [==============================] - 0s 421us/step - loss: 7.0634e-04 - accuracy: 0.4602
Loss: 0.0007063411176204681, Accuracy: 0.46024635434150696


### Step 4: Save and export your model to an HDF5 file

In [28]:
# Set the model's file path
filename = "DLNN_model_1"
file_path = Path('Resources/' + filename + '.h5')

# Export your model to a HDF5 file
nn.save(file_path)


---

## Optimize the neural network model


### Step 1: Define at least three new deep neural network models (resulting in the original plus 3 optimization attempts). With each, try to improve on your first model’s predictive accuracy.

> **Rewind** Recall that perfect accuracy has a value of 1, so accuracy improves as its value moves closer to 1. To optimize your model for a predictive accuracy as close to 1 as possible, you can use any or all of the following techniques:
>
> * Adjust the input data by dropping different features columns to ensure that no variables or outliers confuse the model.
>
> * Add more neurons (nodes) to a hidden layer.
>
> * Add more hidden layers.
>
> * Use different activation functions for the hidden layers.
>
> * Add to or reduce the number of epochs in the training regimen.


### Alternative Model 1 - Deep learning model with 2 hidden layers

In [29]:
# Define the the number of inputs (features) to the model
number_input_features = len(X_train.iloc[0])
print(number_input_features)


25


In [30]:
# Review the number of features
number_input_features

25

In [31]:
# Define the number of neurons in the output layer
number_output_neurons_A1 = 2

In [32]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1_A1 =  int(round((number_input_features + number_output_neurons_A1)/2, 0))
# Review the number hidden nodes in the first layer
hidden_nodes_layer1_A1


14

In [33]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2_A1 =  int(round((hidden_nodes_layer1_A1 + number_output_neurons_A1)/2, 0))

# Review the number hidden nodes in the second layer
hidden_nodes_layer2_A1


8

In [34]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer3_A1 =  int(round((hidden_nodes_layer2_A1 + number_output_neurons_A1)/2, 0))

# Review the number hidden nodes in the second layer
hidden_nodes_layer3_A1


5

In [35]:
# Create the Sequential model instance
nn_A1 = Sequential()

In [36]:
# Add the first hidden layer
nn_A1.add(Dense(units=hidden_nodes_layer1_A1, activation="relu", input_dim=number_input_features))


In [37]:
# Add the second hidden layer
nn_A1.add(Dense(units=hidden_nodes_layer2_A1, activation="relu"))


In [38]:
# Add the second hidden layer
nn_A1.add(Dense(units=hidden_nodes_layer3_A1, activation="relu"))


In [39]:
# Add the output layer to the model specifying the number of output neurons and activation function
nn_A1.add(Dense(units=number_output_neurons_A1, activation="sigmoid"))


In [40]:
# Display the Sequential model summary
nn_A1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 14)                364       
_________________________________________________________________
dense_4 (Dense)              (None, 8)                 120       
_________________________________________________________________
dense_5 (Dense)              (None, 5)                 45        
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 12        
Total params: 541
Trainable params: 541
Non-trainable params: 0
_________________________________________________________________


In [41]:
# Compile the Sequential model
nn_A1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [42]:
# Fit the model using 50 epochs and the training data
model_A1 = nn_A1.fit(X_train_scaled, y_train, epochs=50, verbose=0)


### Alternative Model 2 - Deep learning model with 3 hidden layers

In [43]:
# Define the the number of inputs (features) to the model
number_input_features = len(X_train.iloc[0])
print(number_input_features)


25


In [44]:
# Review the number of features
number_input_features

25

In [45]:
# Define the number of neurons in the output layer
number_output_neurons_A2 = 2

In [46]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1_A2 =  int(round((number_input_features + number_output_neurons_A2)/2, 0))
# Review the number hidden nodes in the first layer
hidden_nodes_layer1_A2


14

In [47]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2_A2 =  int(round((hidden_nodes_layer1_A2 + number_output_neurons_A2)/2, 0))

# Review the number hidden nodes in the second layer
hidden_nodes_layer2_A2


8

In [48]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer3_A2 =  int(round((hidden_nodes_layer2_A1 + number_output_neurons_A2)/2, 0))

# Review the number hidden nodes in the second layer
hidden_nodes_layer3_A2


5

In [49]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer4_A2 =  int(round((hidden_nodes_layer3_A1 + number_output_neurons_A2)/2, 0))

# Review the number hidden nodes in the second layer
hidden_nodes_layer4_A2


4

In [50]:
# Create the Sequential model instance
nn_A2 = Sequential()

In [51]:
# Add the first hidden layer
nn_A2.add(Dense(units=hidden_nodes_layer1_A2, activation="relu", input_dim=number_input_features))


In [52]:
# Add the second hidden layer
nn_A2.add(Dense(units=hidden_nodes_layer2_A2, activation="relu"))


In [53]:
# Add the second hidden layer
nn_A2.add(Dense(units=hidden_nodes_layer3_A2, activation="relu"))


In [54]:
# Add the second hidden layer
nn_A2.add(Dense(units=hidden_nodes_layer4_A2, activation="relu"))


In [55]:
# Add the output layer to the model specifying the number of output neurons and activation function
nn_A2.add(Dense(units=number_output_neurons_A2, activation="sigmoid"))


In [56]:
# Display the Sequential model summary
nn_A2.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 14)                364       
_________________________________________________________________
dense_8 (Dense)              (None, 8)                 120       
_________________________________________________________________
dense_9 (Dense)              (None, 5)                 45        
_________________________________________________________________
dense_10 (Dense)             (None, 4)                 24        
_________________________________________________________________
dense_11 (Dense)             (None, 2)                 10        
Total params: 563
Trainable params: 563
Non-trainable params: 0
_________________________________________________________________


In [57]:
# Compile the Sequential model
nn_A2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [58]:
# Fit the model using 50 epochs and the training data
model_A2 = nn_A2.fit(X_train_scaled, y_train, epochs=50, verbose=0)


#### Alternative Model 3 - Principal Component Analysis

In [59]:
from sklearn.decomposition import PCA


In [60]:
# Create a StandardScaler instance
pca_scaler = StandardScaler()

# Fit the scaler to the features
X_pca_scaler = pca_scaler.fit(X)

# Fit the scaler to the features
X_pca_scaled = X_pca_scaler.transform(X)


In [61]:
# for n in range(2, number_input_features):
n = number_input_features
best_n = n
total_exp_var = 100.0
while total_exp_var > 99.999:
# for n in range(number_input_features, 2, -1):
    pca = PCA(n_components=n, random_state=1)
    X_pca = pca.fit_transform(X_pca_scaled)
    exp_var = pca.explained_variance_ratio_
#     print(f'{exp_var}')
    total_exp_var = 100*exp_var.sum()
    print(f'N = {n:3}: Total explained variance: {total_exp_var:.4f}%')

    if total_exp_var > 99.999:
        best_n = n
    n -= 1

N =  25: Total explained variance: 100.0000%
N =  24: Total explained variance: 100.0000%
N =  23: Total explained variance: 100.0000%
N =  22: Total explained variance: 100.0000%
N =  21: Total explained variance: 99.9999%
N =  20: Total explained variance: 99.9998%
N =  19: Total explained variance: 99.9996%
N =  18: Total explained variance: 99.9993%
N =  17: Total explained variance: 99.9984%


In [62]:
print(best_n)
pca = PCA(n_components=best_n, random_state=1)
X_pca = pca.fit_transform(X_pca_scaled)
exp_var = pca.explained_variance_ratio_
print(f'N = {best_n:3}: Total explained variance: {100*exp_var.sum():.4f}%')


18
N =  18: Total explained variance: 99.9993%


In [63]:
X_pca_train, X_pca_test, y_pca_train, y_pca_test = train_test_split(X_pca, y, random_state=1)

In [64]:
# Define the the number of inputs (features) to the model
number_input_features = best_n
print(number_input_features)


18


In [65]:
# Define the number of neurons in the output layer
number_output_neurons_A3 = 2

In [66]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1_A3 =  int(round((number_input_features + number_output_neurons_A3)/2, 0))
# Review the number hidden nodes in the first layer
hidden_nodes_layer1_A3


10

In [67]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2_A3 =  int(round((hidden_nodes_layer1_A3 + number_output_neurons_A3)/2, 0))

# Review the number hidden nodes in the second layer
hidden_nodes_layer2_A3


6

In [68]:
# Define the number of hidden nodes for the third hidden layer
hidden_nodes_layer3_A3 =  int(round((hidden_nodes_layer2_A3 + number_output_neurons_A3)/2, 0))

# Review the number hidden nodes in the third layer
hidden_nodes_layer3_A3


4

In [69]:
# Create the Sequential model instance
nn_A3 = Sequential()

In [70]:
# Add the first hidden layer
nn_A3.add(Dense(units=hidden_nodes_layer1_A3, activation="relu", input_dim=number_input_features))


In [71]:
# Add the second hidden layer
nn_A3.add(Dense(units=hidden_nodes_layer2_A3, activation="relu"))


In [72]:
# Add the third hidden layer
nn_A3.add(Dense(units=hidden_nodes_layer3_A3, activation="relu"))


In [73]:
# Add the output layer to the model specifying the number of output neurons and activation function
nn_A3.add(Dense(units=number_output_neurons_A3, activation="sigmoid"))


In [74]:
# Display the Sequential model summary
nn_A3.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 10)                190       
_________________________________________________________________
dense_13 (Dense)             (None, 6)                 66        
_________________________________________________________________
dense_14 (Dense)             (None, 4)                 28        
_________________________________________________________________
dense_15 (Dense)             (None, 2)                 10        
Total params: 294
Trainable params: 294
Non-trainable params: 0
_________________________________________________________________


In [75]:
# Compile the Sequential model
nn_A3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [76]:
# Fit the model using 50 epochs and the training data
model_A3 = nn_A3.fit(X_pca_train, y_pca_train, epochs=50, verbose=1)


Epoch 1/50
252/252 [==============================] - 0s 501us/step - loss: 0.3876 - accuracy: 0.2611
Epoch 2/50
252/252 [==============================] - 0s 517us/step - loss: 0.1657 - accuracy: 0.2611
Epoch 3/50
252/252 [==============================] - 0s 493us/step - loss: 0.0708 - accuracy: 0.2611
Epoch 4/50
252/252 [==============================] - 0s 513us/step - loss: 0.0448 - accuracy: 0.2825
Epoch 5/50
252/252 [==============================] - 0s 528us/step - loss: 0.0347 - accuracy: 0.6519
Epoch 6/50
252/252 [==============================] - 0s 513us/step - loss: 0.0287 - accuracy: 0.6730
Epoch 7/50
252/252 [==============================] - 0s 513us/step - loss: 0.0241 - accuracy: 0.6355
Epoch 8/50
252/252 [==============================] - 0s 509us/step - loss: 0.0208 - accuracy: 0.5813
Epoch 9/50
252/252 [==============================] - 0s 548us/step - loss: 0.0172 - accuracy: 0.5246
Epoch 10/50
252/252 [==============================] - 0s 511us/step - loss: 0.014

### Step 2: After finishing your models, display the accuracy scores achieved by each model, and compare the results.

In [77]:
print("Original Model Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=True)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Original Model Results
84/84 [==============================] - 0s 378us/step - loss: 7.0634e-04 - accuracy: 0.4602
Loss: 0.0007063411176204681, Accuracy: 0.46024635434150696


In [78]:
print("Alternative Model 1 Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_A1.evaluate(X_test_scaled, y_test, verbose=True)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Alternative Model 1 Results
84/84 [==============================] - 0s 389us/step - loss: 6.1215e-04 - accuracy: 0.6794
Loss: 0.0006121503538452089, Accuracy: 0.6793579459190369


In [79]:
print("Alternative Model 2 Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_A2.evaluate(X_test_scaled, y_test, verbose=True)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Alternative Model 2 Results
84/84 [==============================] - 0s 385us/step - loss: 1.4719e-05 - accuracy: 0.8335
Loss: 1.4718576494487934e-05, Accuracy: 0.8335199952125549


In [80]:
print("Alternative Model 3 Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_A3.evaluate(X_pca_test, y_pca_test, verbose=True)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Alternative Model 3 Results
84/84 [==============================] - 0s 389us/step - loss: 7.2803e-07 - accuracy: 0.4117
Loss: 7.280316935975861e-07, Accuracy: 0.41172078251838684


### Run different numbers of epochs on best model

In [81]:
# Fit the model using the training data with different numbers of epochs
model_list = []
for n in range(50, 151, 20):
    print(f'Fitting model, epochs: {n}')
    model_list.append(nn_A1.fit(X_train_scaled, y_train, epochs=50, verbose=False))
#     model_A1 = nn_A1.fit(X_train_scaled, y_train, epochs=50, verbose=False)

    # Evaluate the model loss and accuracy metrics using the evaluate method and the test data
    model_loss, model_accuracy = nn_A1.evaluate(X_test_scaled, y_test, verbose=True)
    # Display the model loss and accuracy results
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")    

Fitting model, epochs: 50
84/84 [==============================] - 0s 395us/step - loss: 4.4309e-04 - accuracy: 0.6487
Loss: 0.00044309100485406816, Accuracy: 0.6487495303153992
Fitting model, epochs: 70
84/84 [==============================] - 0s 395us/step - loss: 3.6810e-04 - accuracy: 0.6443
Loss: 0.0003680962836369872, Accuracy: 0.6442702412605286
Fitting model, epochs: 90
84/84 [==============================] - 0s 405us/step - loss: 3.0105e-04 - accuracy: 0.6603
Loss: 0.0003010539512615651, Accuracy: 0.6603209972381592
Fitting model, epochs: 110
84/84 [==============================] - 0s 421us/step - loss: 2.5653e-04 - accuracy: 0.6592
Loss: 0.00025652869953773916, Accuracy: 0.6592012047767639
Fitting model, epochs: 130
84/84 [==============================] - 0s 418us/step - loss: 2.2682e-04 - accuracy: 0.6644
Loss: 0.00022681888367515057, Accuracy: 0.6644270420074463
Fitting model, epochs: 150
84/84 [==============================] - 0s 461us/step - loss: 2.0914e-04 - accurac

In [82]:
# for model in model_list:
#     pass
#     print(model.history)
    # Evaluate the model loss and accuracy metrics using the evaluate method and the test data
#     model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=True)
    # Display the model loss and accuracy results
#     print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")    

### Step 3: Save each of your alternative models as an HDF5 file.


In [83]:
# Set the file path for the first alternative model
filename = "DLNN_model_A1"
file_path_A1 = Path('Resources/' + filename + '.h5')

# Export your model to a HDF5 file
nn_A1.save(file_path_A1)


In [84]:
# Set the file path for the second alternative model
filename = "DLNN_model_A2"
file_path_A2 = Path('Resources/' + filename + '.h5')

# Export your model to a HDF5 file
nn_A2.save(file_path_A2)


In [85]:
# # Set the file path for the third alternative model
filename = "DLNN_model_A3"
file_path_A3 = Path('Resources/' + filename + '.h5')

# # Export your model to a HDF5 file
nn_A3.save(file_path_A3)
